In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/225.8 GB disk)


In [3]:
from ultralytics import YOLO

from IPython.display import display, Image

In [4]:
!yolo task=detect mode=predict model="/content/drive/MyDrive/best.pt"  conf=0.25 source="/content/drive/MyDrive/emb5.png" save=True save_txt=True save_conf=True

2023-08-24 09:57:40.615273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 09:57:44.375845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 168 layers, 11129454 parameters, 0 gradients, 28.5 GFLOPs
image 1/1 /content/drive/MyDrive/emb5.png: 800x544 1 0, 1 1, 1 3, 1 6, 1 7, 1 8, 1 9, 935.3ms
Speed: 6.3ms pre-process, 935.3ms inference, 27.4ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


In [12]:
import math

def calculate_distance(box1, box2):
    x1, y1, _, _ = box1
    x2, y2, _, _ = box2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def reorder_and_calculate_distances(input_file_path, output_file_path):
    with open(input_file_path, 'r') as file:
        lines = file.readlines()

    boxes = []
    for line in lines:
        data = line.strip().split()
        # Assuming the format: <object-class> <x> <y> <width> <height> <confidence>
        class_name = data[0]
        x = float(data[1])
        y = float(data[2])
        width = float(data[3])
        height = float(data[4])
        box = (class_name, x, y, width, height)
        boxes.append(box)

    sorted_boxes = sorted(boxes, key=lambda box: (box[2], box[3]))

    distances = []
    with open(output_file_path, 'w') as output_file:
        for i in range(len(sorted_boxes) - 1):
            distance = calculate_distance(sorted_boxes[i][1:], sorted_boxes[i+1][1:])
            if distance > 0.3:
                print(f"Warning: Distance between boxes {i} and {i+1} is greater than 0.1: {distance}")
            distances.append(distance)

        # Write the sorted boxes in reverse order to the output file
        for box in reversed(sorted_boxes):
            class_name, x, y, width, height = box
            output_file.write(f"{class_name} {x} {y} {width} {height}\n")

    return distances

# Example usage
input_file_path = '/content/drive/MyDrive/emb5.txt'
output_file_path = '/content/drive/MyDrive/emb5_reordered.txt'
distances = reorder_and_calculate_distances(input_file_path, output_file_path)
for distance in distances:
    print(distance)



0.11989501375787064
0.21772384340948972
0.11256388998697588
0.14271262426639064
0.08119192514653166
0.05503367195635775


In [13]:
def create_xml_annotation(image_filename, image_width, image_height, boxes, output_path):
    xml_content = f'''<annotation>
    <folder>FH-OG-OCR-Images</folder>
    <filename>{image_filename}</filename>
    <path>/home/student/Downloads/FH-OG-OCR-Janoschka-Images/FH-OG-OCR-Images/{image_filename}</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>{image_width}</width>
        <height>{image_height}</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    '''

    for class_name, box in boxes:
        x_center, y_center, width, height = box
        xmin = int((x_center - width / 2) * image_width)
        ymin = int((y_center - height / 2) * image_height)
        xmax = int((x_center + width / 2) * image_width)
        ymax = int((y_center + height / 2) * image_height)

        xml_content += f'''<object>
        <name>{class_name}</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{xmin}</xmin>
            <ymin>{ymin}</ymin>
            <xmax>{xmax}</xmax>
            <ymax>{ymax}</ymax>
        </bndbox>
        </object>
        '''

    xml_content += '</annotation>'

    with open(output_path, 'w') as output_file:
        output_file.write(xml_content)

# Read ordered bounding box information with class names from a text file
input_txt_path = '/content/drive/MyDrive/emb5_reordered.txt'
boxes = []
with open(input_txt_path, 'r') as input_file:
    for line in input_file:
        class_name, x_center, y_center, width, height = line.strip().split()
        class_name = int(class_name)
        x_center, y_center, width, height = map(float, (x_center, y_center, width, height))
        boxes.append((class_name, (x_center, y_center, width, height)))

# Example usage
image_filename = 'emb5.png'
image_width = 522
image_height = 800
output_xml_path = '/content/drive/MyDrive/emb5_annotation.xml'

create_xml_annotation(image_filename, image_width, image_height, boxes, output_xml_path)



In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

arr_out = []
arr_result = ['0','1','2','3','4','5','6','7','8','9']
# Define the Monte Carlo Dropout layer
class MonteCarloDropout(tf.keras.layers.Dropout):
    def dropped_inputs(self, inputs, training=None):
        return super().call(inputs, training=training)

# Register the custom layer
tf.keras.utils.get_custom_objects()['MonteCarloDropout'] = MonteCarloDropout

# Load the trained model
model = load_model('/content/drive/MyDrive/modelweightbalanceddata.h5')

In [16]:

import xml.etree.ElementTree as ET
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

def process_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    rois = []
    for obj in root.iter('object'):
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)
        rois.append((xmin, ymin, xmax - xmin, ymax - ymin))

    return rois

def visualize_image_with_rois(image_file, rois):
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    for roi in rois:
        x, y, w, h = roi
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)



# Example usage
xml_file = '/content/drive/MyDrive/emb5_annotation.xml'
image_file = '/content/drive/MyDrive/emb5.png'

rois = process_xml(xml_file)
visualize_image_with_rois(image_file, rois)

# Batch inference
batch_rois = []
for roi in rois:
    x, y, w, h = roi
    roi_image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)[y:y+h, x:x+w]
    roi_image = cv2.resize(roi_image, (64, 64), interpolation=cv2.INTER_AREA)
    roi_image = np.expand_dims(roi_image, axis=-1)
    roi_image = roi_image / 255.0  # Normalize the pixel values

    batch_rois.append(roi_image)

# Convert the batch_rois list to a numpy array
batch_rois = np.array(batch_rois)
# Perform prediction using the model on the batch
predictions = model.predict(batch_rois)

# Initialize a dictionary to store class probabilities
class_probabilities = {}

# Process the predictions
for i, roi in enumerate(rois):
    prediction = predictions[i]
    class_label = arr_result[np.argmax(prediction)]

    if class_label not in class_probabilities:
        class_probabilities[class_label] = []

    class_probabilities[class_label].append(prediction.tolist())

# Print the class probabilities
for class_label, probabilities in class_probabilities.items():
    probabilities = np.mean(probabilities, axis=0)
    probabilities_formatted = [f"{prob:.4f}" for prob in probabilities]
    probabilities_str = ", ".join(probabilities_formatted)
    print(f"Class {class_label} probabilities: {probabilities_str}")


1/1 [==============================] - 1s 714ms/step
Class 7 probabilities: 0.0001, 0.0065, 0.0011, 0.0007, 0.0016, 0.0000, 0.0000, 0.9899, 0.0000, 0.0000
Class 1 probabilities: 0.0011, 0.9330, 0.0004, 0.0000, 0.0036, 0.0000, 0.0001, 0.0617, 0.0001, 0.0001
Class 8 probabilities: 0.0000, 0.0000, 0.0000, 0.0001, 0.0000, 0.0000, 0.0000, 0.0000, 0.9999, 0.0000
Class 0 probabilities: 0.9999, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0001
Class 6 probabilities: 0.0000, 0.0000, 0.0000, 0.0028, 0.0000, 0.0016, 0.9226, 0.0000, 0.0004, 0.0727
Class 9 probabilities: 0.0012, 0.0000, 0.0002, 0.0000, 0.0000, 0.0032, 0.0493, 0.0000, 0.0000, 0.9461
Class 3 probabilities: 0.0000, 0.0000, 0.0000, 0.9992, 0.0000, 0.0005, 0.0000, 0.0000, 0.0001, 0.0000
